In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# Load the HateXplain dataset using HuggingFace datasets library
hatexplain_dataset = load_dataset('hatexplain')

# Function to extract majority label from annotators
def extract_label(annotators):
    labels = annotators['label']
    majority_label = max(set(labels), key=labels.count)
    return majority_label

# Convert the dataset to pandas DataFrame
hatexplain_df = pd.DataFrame(hatexplain_dataset['train'])

# Extract labels
hatexplain_df['label'] = hatexplain_df['annotators'].apply(lambda x: extract_label(x))

# Convert labels to binary (assuming '2' for hatespeech, '0' and '1' for non-hate)
hatexplain_df['label'] = hatexplain_df['label'].apply(lambda x: 1 if x == 2 else 0)

# Combine the text tokens into a single string
hatexplain_df['text'] = hatexplain_df['post_tokens'].apply(lambda x: ' '.join(x))

# Select relevant columns
hatexplain_df = hatexplain_df[['text', 'label']]

# Split the HateXplain data into train and test sets
train_hatexplain_df, test_hatexplain_df = train_test_split(hatexplain_df, test_size=0.2, random_state=42)

# Verify the preprocessing steps
print(train_hatexplain_df.head())
print(test_hatexplain_df.head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


                                                    text  label
12068  it was the intent of the founding fathers and ...      0
12866  if you wanna listen to gay music for gay peopl...      1
3656   not supposed to say the n word if you are whit...      0
8962   aw wook at the widdle faggot twying to shit po...      0
10411  <user> good will be more hindoo than islamoc p...      0
                                                    text  label
13824  currynigger and known race traitor <user> has ...      0
7704                         <user> do you hate yourself      0
8953   why change the game for the nigger assholes if...      0
5344   <user> those are invaders not refugees they ar...      1
12698  <user> the nigger has been prosperous too long...      0


In [ ]:
import openai
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
import torch

# Set up OpenAI API key
openai.api_key = ''

# Function to create a prompt and get response from gpt-3.5-turbo
def gpt3_predict(text):
    prompt = f"Do you think this comment is hate speech?\nComment: {text}\nA. Yes\nB. No"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=10,
        n=1,
        stop=None,
        temperature=0.5
    )
    answer = response.choices[0].message['content'].strip()
    return 1 if 'yes' in answer.lower() else 0

# Function to use GPT-2 from Hugging Face
def gpt2_predict(text, model, tokenizer):
    prompt = f"Do you think this comment is hate speech?\nComment: {text}\nA. Yes\nB. No"
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs['attention_mask'] = torch.ones_like(inputs['input_ids'])  # Explicitly set the attention mask
    outputs = model.generate(**inputs, max_new_tokens=50)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return 1 if 'yes' in answer.lower() else 0

# Initialize GPT-2
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_tokenizer.pad_token_id = gpt2_tokenizer.eos_token_id  # Set pad token to eos token

# Function to use BERT for classification (using a fine-tuned model from Hugging Face)
def bert_predict(text, classifier):
    result = classifier(text)
    return 1 if result[0]['label'] == 'LABEL_1' else 0

# Initialize BERT classifier
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# Function to evaluate models on a given dataset
def evaluate_models(test_df):
    # Lists to store the predictions
    gpt3_predictions = []
    gpt2_predictions = []
    bert_predictions = []

    # Actual labels
    labels = test_df['label'].tolist()

    # Test the models
    for text in test_df['text']:
        gpt3_predictions.append(gpt3_predict(text))
        gpt2_predictions.append(gpt2_predict(text, gpt2_model, gpt2_tokenizer))
        bert_predictions.append(bert_predict(text, classifier))

    # Calculate evaluation metrics
    def evaluate(predictions, labels):
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions)
        recall = recall_score(labels, predictions)
        f1 = f1_score(labels, predictions)
        return accuracy, precision, recall, f1

    gpt3_metrics = evaluate(gpt3_predictions, labels)
    gpt2_metrics = evaluate(gpt2_predictions, labels)
    bert_metrics = evaluate(bert_predictions, labels)

    return gpt3_metrics, gpt2_metrics, bert_metrics

# Evaluate on HateXplain dataset
print("Evaluating on HateXplain dataset")
gpt3_metrics, gpt2_metrics, bert_metrics = evaluate_models(test_hatexplain_df)
print("GPT-3 Metrics: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1 Score: {:.2f}".format(*gpt3_metrics))
print("GPT-2 Metrics: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1 Score: {:.2f}".format(*gpt2_metrics))
print("BERT Metrics: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1 Score: {:.2f}".format(*bert_metrics))


Evaluating on HateXplain dataset


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import load_dataset
import openai
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
import torch

# --- Data Loading and Preprocessing ---
hatexplain_dataset = load_dataset('hatexplain')
hatexplain_df = pd.DataFrame(hatexplain_dataset['train'])
hatexplain_df['label'] = hatexplain_df['annotators'].apply(lambda x: max(set(x['label']), key=x['label'].count))
hatexplain_df['label'] = hatexplain_df['label'].apply(lambda x: 1 if x == 2 else 0)
hatexplain_df['text'] = hatexplain_df['post_tokens'].apply(lambda x: ' '.join(x))
hatexplain_df = hatexplain_df[['text', 'label']]

# --- Subsampling the Data ---
sample_size = 1000
train_hatexplain_df, test_hatexplain_df = train_test_split(hatexplain_df.sample(n=sample_size*2, random_state=42),
                                                          test_size=0.5, random_state=42)

# --- Model Setup ---
openai.api_key =  # Replace with your actual key

gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_tokenizer.pad_token_id = gpt2_tokenizer.eos_token_id

classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# --- Prediction Functions ---
def gpt3_predict(text):
    prompt = f"Do you think this comment is hate speech?\nComment: {text}\nA. Yes\nB. No"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=10,
        n=1,
        stop=None,
        temperature=0.5
    )
    answer = response.choices[0].message['content'].strip()
    return 1 if 'yes' in answer.lower() else 0

def gpt2_predict(text, model, tokenizer):
    prompt = f"Do you think this comment is hate speech?\nComment: {text}\nA. Yes\nB. No"
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs['attention_mask'] = torch.ones_like(inputs['input_ids'])
    outputs = model.generate(**inputs, max_new_tokens=50)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return 1 if 'yes' in answer.lower() else 0

def bert_predict(text, classifier):
    result = classifier(text)
    return 1 if result[0]['label'] == 'LABEL_1' else 0

# --- Evaluation Function ---
def evaluate_models(test_df):
    gpt3_preds, gpt2_preds, bert_preds = [], [], []
    labels = test_df['label'].tolist()

    for text in test_df['text']:
        gpt3_preds.append(gpt3_predict(text))
        gpt2_preds.append(gpt2_predict(text, gpt2_model, gpt2_tokenizer))
        bert_preds.append(bert_predict(text, classifier))

    def evaluate(predictions, labels):
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions)
        recall = recall_score(labels, predictions)
        f1 = f1_score(labels, predictions)
        return accuracy, precision, recall, f1

    gpt3_metrics = evaluate(gpt3_preds, labels)
    gpt2_metrics = evaluate(gpt2_preds, labels)
    bert_metrics = evaluate(bert_preds, labels)

    return gpt3_metrics, gpt2_metrics, bert_metrics

# --- Evaluation ---
print("Evaluating on HateXplain subset")
gpt3_metrics, gpt2_metrics, bert_metrics = evaluate_models(test_hatexplain_df)

print("GPT-3 Metrics: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1 Score: {:.2f}".format(*gpt3_metrics))
print("GPT-2 Metrics: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1 Score: {:.2f}".format(*gpt2_metrics))
print("BERT Metrics: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1 Score: {:.2f}".format(*bert_metrics))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Evaluating on HateXplain subset


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

GPT-3 Metrics: Accuracy: 0.38, Precision: 0.30, Recall: 0.92, F1 Score: 0.45
GPT-2 Metrics: Accuracy: 0.28, Precision: 0.28, Recall: 1.00, F1 Score: 0.43
BERT Metrics: Accuracy: 0.72, Precision: 0.00, Recall: 0.00, F1 Score: 0.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
